# TxtGemma na Vertex AI

Tutorial básico de como fazer um deploy e testar o TxtGemma na Vertex AI.

import json

In [2]:
# Setando o projeto
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

In [3]:
! gcloud storage cp gs://healthai-us/txgemma/templates/tdc_prompts.json tdc_prompts.json

with open("tdc_prompts.json", "r") as f:
    tdc_prompts_json = json.load(f)

Copying gs://healthai-us/txgemma/templates/tdc_prompts.json to file://tdc_prompts.json
  Completed files 1/1 | 750.0kiB/750.0kiB                                      

Average throughput: 188.6MiB/s


## Realizando o Deploy do modelo do Tx Gemma na Vertex AI

In [4]:
ENDPOINT_REGION = "us-west1" 

In [ ]:
import vertexai
from vertexai import model_garden

vertexai.init(project=PROJECT_ID, location=ENDPOINT_REGION)

model = model_garden.OpenModel("google/txgemma@txgemma-9b-predict")
endpoint = model.deploy(
  accept_eula=True,
  machine_type="ct5lp-hightpu-4t",
  serving_container_image_uri="us-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/hex-llm-serve:20241210_2323_RC00",
  endpoint_display_name="google_txgemma-9b-predict-mg-one-click-deploy",
  model_display_name="google_txgemma-9b-predict-1750206418914",
  use_dedicated_endpoint=True,
)

In [ ]:
print(endpoint.name)

In [10]:
ENDPOINT_NAME = endpoint.name

## Utilizando o modelo

In [17]:
# Inicializando a AI Platform

from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [18]:
endpoint_new = aiplatform.Endpoint(
    endpoint_name=ENDPOINT_NAME,
    project=PROJECT_ID,
    location=ENDPOINT_REGION,
)
     

In [20]:
task_name = "BBB_Martins"
input_type = "{Drug SMILES}"
drug_smiles = "CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21"


TDC_PROMPT = tdc_prompts_json[task_name].replace(input_type, drug_smiles)
print("Formatted prompt:\n")
print(TDC_PROMPT)
     

Formatted prompt:

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:


In [21]:
# Prepare request instance using sample prompt for a TDC task
instances = [
    {
        "prompt": TDC_PROMPT,
        "max_tokens": 8,
        "temperature": 0
    },
]

response = endpoint_new.predict(instances=instances)
predictions = response.predictions

print(predictions[0])

B


## Deleção do Endpoint

In [ ]:
input("Pressione Enter para deletar o Endpoint do Índice e o Índice:")

# deletar Index Endpoint
endpoint_new.undeploy_all()
endpoint_new.delete(force=True)
